In [1]:
from nblogger import logger
import numpy as np
import matplotlib.pyplot as plt
import parameters

import theano_shim as shim
import sinn
#import sinn.histories as histories
from sinn.histories import Spiketrain, Series
import sinn.iotools as io
import sinn.analyze as anlz
#from sinn.analyze.heatmap import HeatMap
#import sinn.optimize.gradient_descent as gd

import core
import generate_input

np.set_printoptions(precision=5, suppress=True)
%matplotlib inline

2017-08-27 12:51:41,221 - sinn.analyze.stylelib.color_schemes - WARNING - [monochrome_palette]: value was greater than 1 and was clipped.


In [2]:
input_params = parameters.ParameterSet("input.params")

In [3]:
Ihist = Series.from_raw(io.loadraw(
        core.get_pathname(generate_input.input_subdir, input_params)))

FileNotFoundError: [Errno 2] No such file or directory: 'data/inputs/cecde256f794151b79e1f340fcb8efc357cc8140'

In [4]:
import hashlib

In [5]:
hashlib.sha1(bytes(repr(input_params), 'utf-8')).hexdigest()

'cecde256f794151b79e1f340fcb8efc357cc8140'

In [5]:
hashlib.sha1(bytes(repr(input_params), 'utf-8')).hexdigest()

'96dbb208a0ade5294dbb47fa1ad1c40fe8535c58'

In [6]:
repr(input_params)

"{'t0': 0, 'tn': 20, 'ξ': {'shape': (2,), 'function': 'GaussianWhiteNoise', 'std': 1.0}, 'inputs': ['ξ'], 'dt': 0.0002, 'seed': 314, 'eval': 'ξ'}"

In [3]:
repr(input_params)

"{'dt': 0.0002, 'eval': 'ξ', 'tn': 20, 'ξ': {'std': 1.0, 'function': 'GaussianWhiteNoise', 'shape': (2,)}, 'inputs': ['ξ'], 'seed': 314, 't0': 0}"

In [8]:
input_params.flatten()

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [7]:
sort?

Object `sort` not found.
